<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [1]:
%pip install -Uqqq rich openai==0.27.2 tiktoken wandb tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00


In [3]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)
import wandb
from wandb.integration.openai import autolog

In [4]:
from google.colab import userdata
# Recuperar a chave API do segredo salvo
openai_api_key = userdata.get('OPENAI_API_KEY')

# Definir a chave API como uma variável de ambiente
os.environ['OPENAI_API_KEY'] = openai_api_key
openai.api_key = openai_api_key

In [5]:
# Download files on colab
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/{examples,prompt_template,system_template}.txt

--2024-11-21 00:28:49--  https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40595 (40K) [text/plain]
Saving to: ‘examples.txt’

examples.txt        100%[===================>]  39.64K  --.-KB/s    in 0.008s  

2024-11-21 00:28:49 (5.16 MB/s) - ‘examples.txt’ saved [40595/40595]

--2024-11-21 00:28:49--  https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/prompt_template.txt
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 1054 (1.0K) [text/plain]
Saving to: ‘prompt_template.txt’

prompt_template.txt 100%[===================>]   1.03K  --.-KB/s    in 0s      

2024-11-21 00:28:49 (79.4 M

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [6]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Let's enable W&B autologging to track our experiments.

In [7]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "generation"})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [8]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [9]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [10]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))

generate_and_print(system_prompt, user_prompt)

How do I track individual expenses for different projects within my organization using the W&B software?

Sure! Here is a support question that a user could ask:                                                            

"Hello, I am having trouble adding a new employee to my team in the W&B system. Can you please guide me on how to  
do this correctly?"

Sure, here's a support question a user might ask: "I am having trouble logging into my account despite entering the
correct username and password. Can you help me troubleshoot this issue?"

"Can you provide guidance on how to create custom reports in the W&B platform?"

"Can you provide guidance on how to properly track and manage our company's expenses using the W&B platform?"

# Few Shot

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [11]:
# Test if examples.txt is present, download if not
if not Path("examples.txt").exists():
    !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

In [12]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "my logging doesn't seem to include errors when the training crashes, how do I change the logging level
for wandb logging?"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [13]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: Why do I get  
this error anytime I run wandb.init()? The wandb service process exited with 1. Ensure that `sys.executable` is a  
valid python interpreter. You can override it with the `_executable` setting or with the `WANDB__EXECUTABLE`       
environment variable. I have an config file apart from sweep_config file so when I try to run sweep agent, I am    
getting UnboundLocalError("local variable 'config' referenced before assignment"). How can I log and visualize time
series data in WandB? Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [14]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

How can I debug a "TypeError: unhashable type: 'slice'" error that occurs when trying to use WandB for tracking and
visualization?

Why am I seeing incomplete or missing metrics data in my WandB dashboard even though my code is logging the metrics
properly during training?

How can I use the WandB API to upload custom metrics and visualize them in my project dashboard?

How can I view the detailed logs of my previous runs in WandB?

How can I resolve the error message "The wandb service process exited with 1" when trying to initialize wandb?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [15]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
if not os.path.exists("../docs_sample/"):
  !git clone https://github.com/wandb/edu.git
  !cp -r edu/llm-apps-course/docs_sample ../

Cloning into 'edu'...
remote: Enumerating objects: 4766, done.
remote: Counting objects: 100% (1649/1649), done.
remote: Compressing objects: 100% (725/725), done.
remote: Total 4766 (delta 1185), reused 1195 (delta 912), pack-reused 3117 (from 1)
Receiving objects: 100% (4766/4766), 41.95 MiB | 26.56 MiB/s, done.
Resolving deltas: 100% (2599/2599), done.


In [16]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [17]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[956, 803, 2529, 537, 1206, 4179, 2940, 2093, 365, 1644, 2596]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [18]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [19]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [20]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documentation: ", project="capsules", tags=["debug"])         

...                                                                                                                

if current_loss < threshold: run.tags = run.tags + ("release_candidate",)                                          

                                                                                                                   
                                                                                                                   
 There are also several ways to add tags after runs have been logged to Weights & Biases.                          
                                                                                                                   
 <Tabs                                                                                                             
   defaultValue="publicapi"                                                                                        
   values={[                                                                                                       
     {label: 'Using the Public API', value: 'publicapi'},                                                          
     {label: 'Project Page', value: 'projectpage'},                                                                
     {label: 'Run Page', value: 'runpage'},                                                                        
   ]}>                                                                                                             
   <TabItem value="publicapi">                                                                                     
                                                                                                                   
 After a run is created, you can update tags using [our public API](../../../guides/track/public-api-guide.md) lik 
 so:                                                                                                               
                                                                                                                   
 ```python                                                                                                         
 run = wandb.Api().run("{entity}/{project}/{run-id}"})                                                             
 run.tags.append("tag1")  # you can choose tags based on run data here                                             
 run.update()                                                                                                      
                                                                                                                   

You can read more about how to use the Public API in the ]8;id=991480;../../../ref/README.md\reference documentation]8;;\ or ]8;id=992654;../../../guides/track/public-api-guide.md\guide]8;;\.                         


This method is best suited to tagging large numbers of runs with the same tag or tags.                             

In the ]8;id=109167;../pages/project-page.md#search-for-runs\runs sidebar]8;;\ of the ]8;id=502451;../pages/project-page.md\Project Page]8;;\,  click the table icon in the upper-right.  This will expand the sidebar   
into the full ]8;id=374198;runs-table.md\runs table]8;;\.                                                                                          

Hover over a run in the table to see a checkbox on the left or look in the header row for a checkbox that will     
allow you to select all runs.                                                                                      

Click the checkbox to enable bulk actions. Select the runs to which you'd like to apply your tag(s).               

Click the Tag button ab

Let's generate 3 possible questions:

In [ ]:
generate_and_print(system_prompt, generation_prompt, n=3)

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [ ]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4"

In [ ]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [ ]:
Markdown(system_prompt)

In [ ]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [ ]:
Markdown(prompt_template)

In [ ]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [ ]:
Markdown(user_prompt)

In [ ]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [ ]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None

    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [ ]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

In [ ]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

In [ ]:
wandb.finish()